In [1]:
import json

file = open("./data/clean_adult_video.json","r")
video_data = json.load(file)
file.close()

with open("./data/video_id_list.txt","r") as f:
    video_id_list = f.read().split(',')

In [5]:
corpas = []

for id,video_data in zip(video_id_list,video_data):
    tmp = {"id" : "","content" : ""}
    tmp["id"] = id
    
    tmp["content"] += video_data["title"]
    tmp["content"] += video_data["describe"]
    tmp["content"] += "".join(video_data["review"])

    corpas.append(tmp)

In [7]:
from janome.tokenizer import Tokenizer
from gensim.models.doc2vec import TaggedDocument

m = Tokenizer(wakati=True)

# 分かち書き
from tqdm import tqdm as tq

training_data = []
for document in tq(corpas,total=len(corpas)):
    training_data.append(TaggedDocument(words=[word for word in m.tokenize(document["content"])], tags=[document["id"]]))

100%|██████████| 179875/179875 [1:43:40<00:00, 28.92it/s]  


In [9]:
from gensim.models.doc2vec import Doc2Vec
# トレーニング
trained_data = Doc2Vec(documents= training_data, dm = 1, vector_size=200, window=3, epochs=50,min_count=3, workers=4)
trained_data.save("./data/doc2vec.model")